#### Importing Modules

In [148]:
from dataclasses import dataclass, asdict
from dataclass_csv import DataclassReader
from typing import Optional
from enum import Enum
import pandas as pd

#### Reading the Data

In [149]:
class InfillPattern(Enum):
    GRID = "grid"
    HONEYCOMB = "honeycomb"

In [150]:
class Material(Enum):
    ABS = "abs"
    PLA = "pla"

In [151]:
@dataclass
class ThreeDPrinterData:
    layer_height: float
    wall_thickness: int
    infill_density: int
    infill_pattern: InfillPattern
    nozzle_temperature: int
    bed_temperature: int
    print_speed: int
    material: Material
    fan_speed: int
    roughness: int
    tensile_strength: int
    elongation: float

In [152]:
FILENAME = './3dprinterdata_edited.csv'

In [160]:
def readCSVData(flname):
    with open(flname) as data_csv:
        reader = DataclassReader(data_csv, ThreeDPrinterData)
        reader.map('Tensile Strength').to('tensile_strength')
        reader.map('Elongation').to('elongation')
        data_objects = []
        for row in reader:
            data_objects.append(row)
    
    df = pd.DataFrame(data_objects)
    df['infill_pattern'] = df['infill_pattern'].map(lambda obj: obj.value)
    df['material'] = df['material'].map(lambda obj: obj.value)
    
    return df

In [161]:
df = readCSVData(FILENAME)

In [163]:
df.head()

,layer_height,wall_thickness,infill_density,infill_pattern,nozzle_temperature,bed_temperature,print_speed,material,fan_speed,roughness,tensile_strength,elongation
0,0.02,8,90,grid,220,60,40,abs,0,25,18,1.2
1,0.02,7,90,honeycomb,225,65,40,abs,25,32,16,1.4
2,0.02,1,80,grid,230,70,40,abs,50,40,8,0.8
3,0.02,4,70,honeycomb,240,75,40,abs,75,68,10,0.5
4,0.02,6,90,grid,250,80,40,abs,100,92,5,0.7
